<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/main/MVA2024_ex07notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2024 ex07notebookB

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2024

---
## 演習課題
---

<b><font color="#ff0000">
注意:
今回の notebook の中には，コードセルを実行すると問題の解答が表示されるようになっている箇所があります．
</font>
</b>



In [ ]:
# いろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# statsmodels の PCA クラス
from statsmodels.multivariate.pca import PCA

# 「解答」を示す際に文字列を復号するのに使う
import base64
# 復号した文字列を Markdown 形式で（数式は LaTeX でフォーマットして）表示
from IPython.display import display, Markdown

---
### 国数英データの主成分分析

前回の notebookB に出てきたのと同じ，「国語」「数学」「英語」3科目の点数を集めたデータに主成分分析を適用する実験を行い，今回の notebookA で学んだことの一部を確認しましょう．


#### データを準備する

次のコードセルを実行すると，データを入手して前処理を行います．

In [ ]:
# 国数英
dfJME = pd.read_table('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/jme.txt', sep=' ', names=['国語', '数学', '英語'])
dfJME

# NumPy array にする
Xorg = dfJME.to_numpy()
print('Xorg の最初の5行')
print(Xorg[:5, :], Xorg.shape)
print()
# 平均を差し引いたものを X とする
XJME_mean = np.mean(Xorg, axis=0)
XJME = Xorg - XJME_mean
N_JME, D_JME = XJME.shape
print('XJME の最初の5行')
print(XJME[:5, :], XJME.shape)

#### 元のデータを可視化する

次のコードセルを実行すると，変数 `XJME` に格納されたデータ（元のデータから変数ごとの平均を差し引いたもの）を可視化します．

In [ ]:
# 平均を 0 にしたデータのヒストグラムと散布図
data = XJME
fig, ax = plt.subplots(2, 3, figsize=(9, 6))
xmin, xmax = -30, 30
xList = [0, 0, 1]
yList = [1, 2, 2]
labelList = ['Japanese', 'Math', 'English']
cor = np.corrcoef(data.T) # 相関係数
for j in range(3):
    # 上の行
    ax[0, j].hist(data[:, j], bins=7)
    ax[0, j].set_xlim(xmin, xmax)
    ax[0, j].set_ylim(0, 10)
    ax[0, j].axvline(0, color='gray')
    ax[0, j].set_xlabel(labelList[j])
    ax[0, j].set_ylabel('frequency')
    # 下の行
    ix, iy = xList[j], yList[j]
    ax[1, j].scatter(data[:, ix], data[:, iy], marker='.')
    ax[1, j].set_xlim(xmin, xmax)
    ax[1, j].set_ylim(xmin, xmax)
    ax[1, j].axhline(0, color='gray')
    ax[1, j].axvline(0, color='gray')
    ax[1, j].set_xlabel(labelList[ix])
    ax[1, j].set_ylabel(labelList[iy])
    ax[1, j].set_aspect('equal')
    ax[1, j].set_title(f'cor = {cor[ix, iy]:.3f}')
fig.tight_layout()
plt.show()

上の行は，3つの変数（国語，数学，英語）のヒストグラムです．下の行では，3つのうち2つの変数を選んで散布図を描いています．散布図の上の `cor = ` というところは，2つの変数間の相関係数を表示しています．

3つのヒストグラムからは，3つの変数の分散にそれほど大きな違いはないことが分かります．

**問題**

散布図と相関係数の値から読み取れることを述べなさい．まずは自分で考えてから，次のセルを実行してね．


In [ ]:
Q = b'CuOBqeOBrjLlpInmlbDjga7plpPjgavjgoLmraPjga7nm7jplqLjgYzjgYLjgovvvI7nibnjgavvvIzlm73oqp7jgajoi7Hoqp7vvIzmlbDlrabjgajoi7Hoqp7jga7plpPjgavlvLfjgYTmraPjga7nm7jplqLjgYzjgYLjgovvvI4K'
display(Markdown(base64.b64decode(Q).decode('utf-8')))

#### 主成分分析を実行する

次のセルを実行すると，`XJME` の分散共分散行列を求めて変数 `Vx` に格納し，その固有値と固有ベクトルを計算します．

In [ ]:
# Xの共分散行列とその対角和
Vx = XJME.T @ XJME / N_JME
print()
print('Vx = ')
print(Vx, Vx.shape)
print(f'(Xの総分散) = (Vxの対角和) = {np.trace(Vx):.2f}')

# Vx の固有値・固有ベクトル （U[h]がh番目の固有ベクトル）
U, eval, Vt = np.linalg.svd(Vx)
print()
print('eval = ', eval)
print(f'sum(eval) = {np.sum(eval):.2f}')
print()
print('U = ')
print(U, U.shape)


**問題**

上記の `eval` は `Vx` の固有値を，`sum(eval)` はそれらの和を表している．上の実行結果から読み取れることとして，次の文中の箱に入る語や数を答えなさい．

- `Vx` の全ての固有値の和は，$\fbox{ア}$ に等しい
- 第1,2,3主成分の分散はそれぞれ $\fbox{イ}$，$\fbox{ウ}$，$\fbox{エ}$ である．

In [ ]:
Q = b'Ci0g44KiOiDjgIxY44Gu57eP5YiG5pWj44CN44GC44KL44GE44Gv44CMVngg44Gu5a++6KeS5ZKM44CNCi0g44KkOiAxNDgg77yI56ysMeS4u+aIkOWIhuOBruWIhuaVo+OBr++8jFZ444Gu5pyA5aSn44Gu5Zu65pyJ5YCk44Gr562J44GX44GE77yJCi0g44KmOiAzMC42Ci0g44KoOiAzLjYwCg=='
display(Markdown(base64.b64decode(Q).decode('utf-8')))

次のセルを実行すると，`Vx` の固有ベクトルをならべた行列 `U` を用いて `XJME` を変換し，その値を `YJME` に格納します．
`YJME` の $h$ 列目には，第$h$主成分スコアの値が入ります（$h = 1, 2, 3$）．

In [ ]:
YJME = XJME @ U
print(YJME.shape)

#### 変換後のデータを可視化する

次のコードセルを実行すると，変数 `YJME` に格納されたデータを可視化します．

In [ ]:
# 主成分分析によって変換したデータのヒストグラムと散布図
data = YJME
fig, ax = plt.subplots(2, 3, figsize=(9, 6))
xmin, xmax = -30, 30
xList = [0, 0, 1]
yList = [1, 2, 2]
labelList = ['PC1', 'PC2', 'PC3']
cor = np.corrcoef(data.T) # 相関係数
for j in range(3):
    # 上の行
    ax[0, j].hist(data[:, j], bins=7)
    ax[0, j].set_xlim(xmin, xmax)
    ax[0, j].set_ylim(0, 10)
    ax[0, j].axvline(0, color='gray')
    ax[0, j].set_xlabel(labelList[j])
    ax[0, j].set_ylabel('frequency')
    # 下の行
    ix, iy = xList[j], yList[j]
    ax[1, j].scatter(data[:, ix], data[:, iy], marker='.')
    ax[1, j].set_xlim(xmin, xmax)
    ax[1, j].set_ylim(xmin, xmax)
    ax[1, j].axhline(0, color='gray')
    ax[1, j].axvline(0, color='gray')
    ax[1, j].set_xlabel(labelList[ix])
    ax[1, j].set_ylabel(labelList[iy])
    ax[1, j].set_aspect('equal')
    ax[1, j].set_title(f'cor = {cor[ix, iy]:.3f}')
fig.tight_layout()
plt.show()

グラフの `PC1`, `PC2`, `PC3` は，第1，第2，第3主成分を表します．
上の行は，主成分スコアのヒストグラムです．下の行では，3つの主成分のうち2つを選んで散布図を描いています．散布図の上の `cor = ` というところは，それらの間の相関係数を表示しています．


**問題**

主成分スコアのヒストグラムを見て，主成分の分散について分かることを述べなさい．「分散共分散行列の固有値」という語を使うこと．思いつかない場合，notebookAの「変換後のデータの性質」をもう一度読み返すこと．

In [ ]:
Q = b'CuS4u+aIkOWIhuWIhuaekOOBp+OBr++8jOesrCAkaCQg5Li75oiQ5YiG44Gu5YiG5pWj44Gv77yM5YWD44Gu44OH44O844K/44Gu5YiG5pWj5YWx5YiG5pWj6KGM5YiX44Gu5Zu65pyJ5YCk44Gu44GG44GhICRoJCDnlarnm67jgavlpKfjgY3jgarjgoLjga7jgavnrYnjgZfjgY/jgarjgovvvI4K44GX44Gf44GM44Gj44Gm77yM5Li75oiQ5YiG44Gu55Wq5Y+344GM5bCP44GV44GE44KC44Gu44G744Gp5YiG5pWj44GM5aSn44GN44GP44Gq44KL77yO44OS44K544OI44Kw44Op44Og44KS6KaL44KL44Go44Gd44Gu44Go44GK44KK44Gr44Gq44Gj44Gm44GE44KL77yOCg=='
display(Markdown(base64.b64decode(Q).decode('utf-8')))

**問題**

散布図と相関係数の値から読み取れることを述べなさい．
思いつかない場合，notebookAの「変換後のデータの性質」をもう一度読み返すこと．

In [ ]:
Q = b'CuS4u+aIkOWIhuWIhuaekOOBq+OCiOOCi+WkieaPm+W+jOOBruODh+ODvOOCv+OBp+OBr++8jOWkieaVsOmWk+OBruWFseWIhuaVo+OBiuOCiOOBs+ebuOmWouS/guaVsOOBjCAwIOOBq+OBquOCi+OBk+OBqOOBjOOCj+OBi+OBo+OBpuOBhOOCi++8jgrjgrDjg6njg5Xjga7kuIrjgavooajnpLrjgZXjgozjgovjganjga4y44Gk44Gu5Li75oiQ5YiG44Gu6ZaT44Gu55u46Zai5L+C5pWw44KC77yM56K644GL44GrIDAg44Gr44Gq44Gj44Gm44GE44KL77yOCuaVo+W4g+Wbs+OCkuimi+OBpuOCgu+8jDLjgaTjga7kuLvmiJDliIbjga7plpPjgavnm7jplqLjgYzjgarjgYTjgZPjgajjgYzjgo/jgYvjgovvvI4K44GT44Gu5pWj5biD5Zuz44Gn44Gv57im6Lu444KI44KK44KC5qiq6Lu444Gu5pa544GM55Wq5Y+344Gu5bCP44GV44GE5Li75oiQ5YiG44Gq44Gu44Gn77yM54K544Gu5bqD44GM44KK44Gv57im6Lu444KI44KK44KC5qiq6Lu444Gu5pa544GM5aSn44GN44GP44Gq44Gj44Gm44GE44KL77yOCg=='
display(Markdown(base64.b64decode(Q).decode('utf-8')))

---
### 外食支出金額データの主成分分析

#### データを準備する

前回の notebookB に出てきたのと同じ外食支出金額のデータ（変数の数は9）の主成分分析をやりましょう．

In [ ]:
# データを扱うためのプログラムを入手
!wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/DiningOutSpendingData.py
# それをインポートして実行
from DiningOutSpendingData import DiningOutSpendingData
dosd = DiningOutSpendingData(dropcolumns=True)
dosd.info()
# DataFrame を入手して最初の5件を表示
dfDining = dosd.getDataFrame()
dfDining.head(5)

変数 `XDining` にデータを格納します．

In [ ]:
# NumPy array を作る
XDining = dosd.getArray()
N_Dining, D_Dining = XDining.shape
print(XDining.shape)

#### 主成分分析を実行する

次のセルを実行すると，statsmodels を使って主成分分析を実行します．以前と同様，データを標準化したものを用います（相関行列に対する主成分分析）．

In [ ]:
# statsmodels で PCA
pca = PCA(XDining, ncomp=None, standardize=True, demean=True, normalize=False)
# 分散共分散行列の固有値
eval = pca.eigenvals
print('固有値 = ', eval)

求めた固有値を使って，寄与率および累積寄与率を求めてみましょう．

In [ ]:
# 寄与率と累積寄与率
c = eval / np.sum(eval)
print('寄与率 = ', c)
cc = np.cumsum(c)
print('累積寄与率 = ', cc)

# それらのグラフ
fig, ax = plt.subplots(figsize=(6, 4))
xv = np.arange(D_Dining)+1
ax.bar(xv, c, label='contribution ratio')
ax.plot(xv, cc, marker='o', label='cumulative CR', color='red')
ax.set_xticks(xv)
ax.legend()
plt.show()

棒グラフが寄与率，折れ線グラフが累積寄与率を表します．横軸は主成分の番号です．

**問題**

次の文は，上記の結果を説明したものです．箱の中に入る数を答えなさい．
分からなければ，notebookAの「寄与率と累積寄与率」をもう一度読み返すこと．

- 第1主成分は，元のデータの総分散の $\fbox{カ}$ 割強を表す．
- 第2主成分は，元のデータの総分散の $\fbox{キ}$ 割弱を表す．
- このデータを主成分分析によって 2 次元に次元削減したものは，元のデータの総分散の $\fbox{ク}$ 割強を保っている．
- 主成分分析によってこのデータの総分散の 8 割以上が保たれるように次元削減するには，削減後の次元数を $\fbox{ケ}$ 以上にする必要がある．

In [ ]:
Q = b'CuOCqyA9ICA0LCDjgq0gPSAyLCDjgq8gPSA2LCDjgrEgPSA0Cg=='
display(Markdown(base64.b64decode(Q).decode('utf-8')))

次のコードセルを実行すると，正規化した主成分負荷量の値を表示します（注）．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: ここでは標準化したデータに対して主成分分析を適用している（相関行列を用いた主成分分析を行っている）ため，正規化した主成分負荷量は，データの分散共分散行列（今の場合これは相関行列に等しい）の固有ベクトルの値に一致します（notebookA 「主成分負荷量」の注釈参照）．
</span>

In [ ]:
# 正規化した主成分負荷量
lf = pca.loadings
pd.options.display.precision = 3
dfLF = pd.DataFrame(lf.T, columns=dfDining.columns[2:], index=[ f'第{i+1}主成分' for i in range(D_Dining)])
dfLF

第1，第2，第3主成分の（正規化した）主成分負荷量をグラフにすると，次のようになります．

In [ ]:
# 正規化した主成分負荷量の可視化
fig, ax = plt.subplots(figsize=(6, 4))
xv = np.arange(D_Dining)+1
ax.plot(xv, lf[:, 0], label='PC1', marker='o')
ax.plot(xv, lf[:, 1], label='PC2', marker='o')
ax.plot(xv, lf[:, 2], label='PC3', marker='o')
ax.axhline(0, color='gray')
ax.set_xticks(xv)
ax.set_ylim(-1, 1)
ax.legend()
plt.show()

横軸の番号は，元のデータの変数の番号に対応しています．1 は「日本そば・うどん」，2は「中華そば」...です．`PC1` は第1主成分の（正規化した）主成分負荷量，`PC2` は第2主成分の...です．

**問題**

第1主成分の（正規化された）主成分負荷量の値から読み取れることを述べなさい．
まずは自分で考えてから，次のセルを実行してね．

In [ ]:
Q = b'CuesrDHkuLvmiJDliIbjga7mraPopo/ljJbjgZXjgozjgZ/kuLvmiJDliIbosqDojbfph4/jga/vvIzjgIzkuK3oj6/jgZ3jgbDjgI3jgavlr77jgZnjgovlgKTjgYwgMCDjgavov5HjgYTjgYzvvIzjgZ3jgozku6XlpJbjga7lpInmlbDjgavlr77jgZfjgabjga/lkIzjgZjjgojjgYbjgarmraPjga7lgKTjgpLjgajjgaPjgabjgYTjgovvvI4K44GT44KM44Gv77yM44CM5Lit6I+v44Gd44Gw44CN5Lul5aSW44Gu44GZ44G544Gm44Gu5aSJ5pWw44Gv77yM56ysMeS4u+aIkOWIhuOBq+WvvuOBl+OBpuWQjOeoi+W6puOBruato+OBruWvhOS4juOCkuOBl+OBpuOBhOOCi++8iOOBneOBruWkieaVsOOBruWApOOBjOato+OBquOCieOBsOS4u+aIkOWIhuOCueOCs+OCouOBjOWkp+OBjeOBj+OBquOCi++8ieOBk+OBqOOCkuaEj+WRs+OBmeOCi++8jgrjgZfjgZ/jgYzjgaPjgabvvIznrKwx5Li75oiQ5YiG44Gv77yM44CM5aSW6aOf5pSv5Ye65YWo5L2T44Gu5aSa44GV77yI44Gf44Gg44GX5Lit6I+v44Gd44Gw44KS6Zmk44GP77yJ44CN44KS6KGo44GZ6YeP44Go6Kej6YeI44Gn44GN44KL77yOCg=='
display(Markdown(base64.b64decode(Q).decode('utf-8')))

**問題**

第2主成分の（正規化された）主成分負荷量の値から読み取れることを述べなさい．
まずは自分で考えてから，次のセルを実行してね．

In [ ]:
Q = b'CuesrDLkuLvmiJDliIbjga7mraPopo/ljJbjgZXjgozjgZ/kuLvmiJDliIbosqDojbfph4/jga7jgYbjgaHvvIzntbblr77lgKTjga7lsI/jgZXjgYQ044Gk44Gr5a++5b+c44GZ44KL5aSJ5pWw44Gv54Sh6KaW44Gn44GN44KL44Go6ICD44GI44KL44Go77yMCgotIOOAjOODj+ODs+ODkOODvOOCrOODvOOAjeOAjOWWq+iMtuS7o+OAjeOBr+esrDLkuLvmiJDliIbjgavmraPjga7lr4TkuI7jgpLjgZfjgabjgYTjgosKLSDjgIzml6XmnKzjgZ3jgbDjg7vjgYbjganjgpPjgI3jgIzkuK3oj6/jgZ3jgbDjgI3jgIzjgZnjgZfvvIjlpJbpo5/vvInjgI3jga/nrKwy5Li75oiQ5YiG44Gr6LKg44Gu5a+E5LiO44KS44GX44Gm44GE44KLCgrjgajjgYTjgYjjgovvvI7jgZfjgZ/jgYzjgaPjgabvvIznrKwy5Li75oiQ5YiG44K544Kz44Ki44GM5aSn44GN44GE5biC44Gv77yM44CM44OP44Oz44OQ44O844Ks44O844CN44CM5Zar6Iy25Luj44CN44G444Gu5pSv5Ye644GM55u45a++55qE44Gr5aSa44GP77yM44CM5pel5pys44Gd44Gw44O744GG44Gp44KT44CN44CM5Lit6I+v44Gd44Gw44CN44CM44GZ44GX77yI5aSW6aOf77yJ44CN44G444Gu5pSv5Ye644GM55u45a++55qE44Gr5bCR44Gq44GE44Go44GE44GI44KL77yOCumAhuOBq++8jOesrDLkuLvmiJDliIbjga7lsI/jgZXjgYTluILjga/vvIzjgIzjg4/jg7Pjg5Djg7zjgqzjg7zjgI3jgIzllqvojLbku6PjgI3jgbjjga7mlK/lh7rjgYznm7jlr77nmoTjgavlsJHjgarjgY/vvIzjgIzml6XmnKzjgZ3jgbDjg7vjgYbjganjgpPjgI3jgIzkuK3oj6/jgZ3jgbDjgI3jgIzjgZnjgZfvvIjlpJbpo5/vvInjgI3jgbjjga7mlK/lh7rjgYznm7jlr77nmoTjgavlpJrjgYTjgajjgYTjgYjjgovvvI4K'
display(Markdown(base64.b64decode(Q).decode('utf-8')))

#### 変換後のデータを可視化する

主成分負荷量の考察をふまえて，元のデータを2次元に次元削減したものの散布図を観察しよう．

In [ ]:
# 主成分スコアを求める
YDining = pca.scores[:, :2]
print(YDining.shape)

# Y の散布図
fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(YDining[:, 0], YDining[:, 1])
ax.scatter(YDining[dosd.kinki5(), 0], YDining[dosd.kinki5(), 1], label='kinki5')
ax.scatter(YDining[dosd.tohoku6(), 0], YDining[dosd.tohoku6(), 1], label='tohoku6')
ax.set_xlim(-7.5, 7.5)
ax.set_ylim(-7.5, 7.5)
ax.set_aspect('equal')
ax.axhline(0, color='gray') # x軸
ax.axvline(0, color='gray') # y軸
ax.legend()
for n in range(YDining.shape[0]):
    plt.annotate(f'{n}', (YDining[n, 0]+0.1, YDining[n, 1]+0))
plt.show()

**問題**

この散布図を見て，次の番号の都市の外食支出の傾向を述べなさい：

- 20番
- 46番
- 16番
- `kinki5` （近畿5府県）
- `tohoku6` （東北6県）

次のコードセルを実行すると，番号と都市名の対応表が表示されます．

In [ ]:
# 図の各点に付された番号と県庁所在市名との対応表
dfDining['City']